# Projet 4

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from multiprocessing import Pool

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

Exemple `URL_PAGE2` doit retourner :

<pre>{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
</pre>

In [14]:
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    
    soup = BeautifulSoup(requests.get(url).content.decode('utf-8'))

    dicp = {
        divt.find('h2').text:{
            'price': divt.find('span', attrs={'class': 'pricing-table-price'}).text.split()[0], 
            'storage': divt.findAll('li')[3].text.split()[0], 
            'databases': int(divt.findAll('li')[4].text.split()[0]) }
        for divt in soup.findAll('div', attrs={'class': 'pricing-table'})}

    return dicp

In [15]:
get_prices_from_url(URL_PAGE2)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}

In [23]:
l = [['personal', 12, 'BG'], ['Small', 15, 'BD']]

d = {
    x[0] : { 'price' : x[1], 'storage' : x[2] } for x in l
}
d

{'personal': {'price': 12, 'storage': 'BG'},
 'Small': {'price': 15, 'storage': 'BD'}}

Ecrire une fonction qui extrait des informations sur une bière de beowulf.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33 

La fonction doit retourner :
<pre>
{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}
</pre>

In [25]:
url = "https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33"

def extract_beer_infos(url):
    infos = {'name': None, 'note': None, 'price': None,'volume': None,}
    soup = BeautifulSoup(requests.get(url).content.decode('utf-8'))

    bearinf =soup.find('div', attrs={'class': 'row detail-beer-variation'})
    
    infos['name']=bearinf.find('h1').text
    infos['note']=int(bearinf.find('div', attrs={'class':'stars'}).attrs['data-percent'])
    infos['price']=float(soup.find('span', attrs={'class':'price'}).text.split()[0].replace(',','.'))
    infos['volume']=int(bearinf.find('span').text.split()[-2])

    return infos

extract_beer_infos(url)

{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}

Cette URL retourne un JSON avec une liste de bières :

In [38]:
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

Ecrire une fonction qui prend l'argument cet URL retourne les informations sur une liste de bière via l'API de beowulf.

Cette fonction doit retourner la liste des informations obtenues par la fonction `extract_beer_infos()` définie ci-dessus.

Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.

Exemple de retour :

<pre>[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
    {'name': 'La Lager Sans Gluten de Vézelay', 'note': 60, 'price': 38.99, 'volume': 25},
    {'name': 'Brasserie De Sutter Brin de Folie', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'La Cristal IPA du Mont Blanc', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'La Parisienne Le Titi Parisien', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
    {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
    {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
    {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]
</pre>

In [39]:

from multiprocessing import Pool

def extract_beer_list_infos(url):
    
    datab = requests.get(url).json()
    
    beerp = ['https://www.beerwulf.com' + item['contentReference'] for item in datab['items']]
    
    beers = [extract_beer_infos(url) for url in beerp]
    
    return beers

extract_beer_list_infos(URL_BEERLIST_FRANCE)

[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
 {'name': 'La Lager Sans Gluten de Vézelay',
  'note': 60,
  'price': 38.99,
  'volume': 25},
 {'name': 'Brasserie De Sutter Brin de Folie',
  'note': 70,
  'price': 44.99,
  'volume': 33},
 {'name': 'La Cristal IPA du Mont Blanc',
  'note': 70,
  'price': 44.99,
  'volume': 33},
 {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
 {'name': 'La Parisienne Le Titi Parisien',
  'note': 70,
  'price': 38.99,
  'volume': 33},
 {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
 {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
 {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
 {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
 {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]

In [40]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        # We should have 11 French beers:
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 11)
        # All of them are 25cl or 33cl:
        for beer in infos:
            self.assertIn(beer['volume'], [25, 33])

            
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [41]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Lesson4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Lesson4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 5.207s

OK
